In [2]:
from IPython.display import display
from one_locus_two_alleles_app import OneLociTwoAllelesSimulationApp
display(OneLociTwoAllelesSimulationApp(allow_selection=False, allow_mutation=False, allow_selfing=False))

OneLociTwoAllelesSimulationApp(children=(HBox(children=(VBox(children=(Label(value='Genotypic proportions'), F…